In [1]:
# Install required libraries
!pip install sentence-transformers usearch torch datasets matplotlib scikit-learn numpy

# Import necessary libraries (will be used across cells)
import numpy as np
from sentence_transformers import SentenceTransformer
from usearch.index import Index
from torch.utils.data import DataLoader
from sentence_transformers import losses, InputExample
import matplotlib.pyplot as plt
from datasets import load_dataset

In [2]:
# search_engine.py
class CodeSearchEngine:
    def __init__(self, model_name='sentence-transformers/all-MiniLM-L6-v2', metric='cos', dimensions=384):
        self.model = SentenceTransformer(model_name)
        self.index = Index(ndim=dimensions, metric=metric, dtype=np.float32)
        self.doc_ids = []

    def index_documents(self, documents):
        """Index a list of code snippet documents."""
        embeddings = self.model.encode(documents, normalize_embeddings=True, show_progress_bar=True)
        for i, emb in enumerate(embeddings):
            self.index.add(i, emb)
            self.doc_ids.append(documents[i])  # Store original docs

    def search(self, query, top_k=10):
        """Search for top-k similar code snippets."""
        query_emb = self.model.encode([query], normalize_embeddings=True)
        matches = self.index.search(query_emb[0], top_k)
        results = [(self.doc_ids[match.id], match.score) for match in matches]
        return results

In [3]:
# evaluation.py
import numpy as np
from scipy.stats import rankdata

def recall_at_k(relevant, predicted, k=10):
    """Recall@K: Fraction of relevant in top-K."""
    top_k = predicted[:k]
    return len(set(relevant) & set(top_k)) / len(relevant) if relevant else 0

def mrr_at_k(relevant, predicted, k=10):
    """MRR@K: Mean reciprocal rank in top-K."""
    top_k = predicted[:k]
    for rank, item in enumerate(top_k, 1):
        if item in relevant:
            return 1 / rank
    return 0

def ndcg_at_k(relevant, predicted, k=10):
    """nDCG@K (binary relevance)."""
    top_k = predicted[:k]
    rel = [1 if i in relevant else 0 for i in top_k]
    dcg = sum(r / np.log2(i + 1) for i, r in enumerate(rel))
    idcg = sum(1 / np.log2(i + 1) for i in range(min(k, len(relevant))))
    return dcg / idcg if idcg > 0 else 0

def evaluate_search(engine, test_queries, ground_truth, k=10):
    """Evaluate on list of (query, relevant_doc_ids)."""
    recalls, mrrs, ndcgs = [], [], []
    for query, rel_ids in zip(test_queries, ground_truth):
        results = engine.search(query, k)
        predicted = [doc for doc, _ in results]  # Assume doc_ids match
        recalls.append(recall_at_k(rel_ids, predicted, k))
        mrrs.append(mrr_at_k(rel_ids, predicted, k))
        ndcgs.append(ndcg_at_k(rel_ids, predicted, k))
    return np.mean(recalls), np.mean(mrrs), np.mean(ndcgs)

In [4]:
# evaluation.py
import numpy as np
from scipy.stats import rankdata

def recall_at_k(relevant, predicted, k=10):
    """Recall@K: Fraction of relevant in top-K."""
    top_k = predicted[:k]
    return len(set(relevant) & set(top_k)) / len(relevant) if relevant else 0

def mrr_at_k(relevant, predicted, k=10):
    """MRR@K: Mean reciprocal rank in top-K."""
    top_k = predicted[:k]
    for rank, item in enumerate(top_k, 1):
        if item in relevant:
            return 1 / rank
    return 0

def ndcg_at_k(relevant, predicted, k=10):
    """nDCG@K (binary relevance)."""
    top_k = predicted[:k]
    rel = [1 if i in relevant else 0 for i in top_k]
    dcg = sum(r / np.log2(i + 1) for i, r in enumerate(rel))
    idcg = sum(1 / np.log2(i + 1) for i in range(min(k, len(relevant))))
    return dcg / idcg if idcg > 0 else 0

def evaluate_search(engine, test_queries, ground_truth, k=10):
    """Evaluate on list of (query, relevant_doc_ids)."""
    recalls, mrrs, ndcgs = [], [], []
    for query, rel_ids in zip(test_queries, ground_truth):
        results = engine.search(query, k)
        predicted = [doc for doc, _ in results]  # Assume doc_ids match
        recalls.append(recall_at_k(rel_ids, predicted, k))
        mrrs.append(mrr_at_k(rel_ids, predicted, k))
        ndcgs.append(ndcg_at_k(rel_ids, predicted, k))
    return np.mean(recalls), np.mean(mrrs), np.mean(ndcgs)

In [5]:
# Manual Dataset Loading
from datasets import load_dataset

# Load the uploaded dataset
dataset = load_dataset("json", data_files="/content/gpt4o_augment_query_code_pairs.json", split="train")

# Inspect the dataset to confirm structure
print(dataset)
print("Columns available:", dataset.column_names)

# Prepare training examples (assuming 'query' and 'code' fields)
train_examples = [InputExample(texts=[q, c], label=1.0) for q, c in zip(dataset['query'], dataset['code'])]
print(f"Number of training examples: {len(train_examples)}")

Dataset({
    features: ['pair-idx', 'query-idx', 'query', 'code-idx', 'code', 'label'],
    num_rows: 108896
})
Columns available: ['pair-idx', 'query-idx', 'query', 'code-idx', 'code', 'label']
Number of training examples: 108896


In [ ]:
# fine_tune.py
from google.colab import drive
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"

# Mount Google Drive (skip if already mounted)
try:
    drive.mount('/content/drive')
except ValueError:
    print("Drive already mounted; proceeding without remount.")

# Use the manually loaded dataset (assume loaded from previous cell)
train_examples = [InputExample(texts=[q, c], label=1.0) for q, c in zip(dataset['query'], dataset['code'])]

# Initialize and fine-tune model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model)

# Fine-tune with loss tracking
loss_history = []
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    warmup_steps=100,
    output_path='/content/drive/MyDrive/fine_tuned_cosqa',
    callback=lambda m, l: loss_history.append(l.item()) if l.item() is not None else None
)

# Plot loss
plt.plot(loss_history)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Mean Loss During Fine-Tuning')
plt.show()

# Save model to Google Drive
model.save('/content/drive/MyDrive/fine_tuned_cosqa')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
